# Query Data with AWS Data Wrangler


In [1]:
%store -r ingest_create_athena_table_csv_passed


In [2]:
try:
    ingest_create_athena_table_csv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [3]:
print(ingest_create_athena_table_csv_passed)


True


In [4]:
if not ingest_create_athena_table_csv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


## Setup

In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [6]:
import awswrangler as wr


### Query the Glue Catalog (ie. Hive Metastore)


In [7]:
database_name = "module2_aws"
table_name_csv = "music_csv2"

In [8]:
for table in wr.catalog.get_tables(database="module2_aws"):
    print(table["Name"])

music_csv
music_csv1
music_csv2


### Query from Athena


In [9]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM {}".format(table_name_csv), database=database_name)

CPU times: user 705 ms, sys: 104 ms, total: 810 ms
Wall time: 5.23 s


In [10]:
df.head()

,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,1GN7Kn9Hr4pOpVVxBL1ZUD,Chris Tomlin;Lauren Daigle,Noel - Live,0,257720,False,0.349,0.261,10,-9.478,0,0.0306,0.74800,0.000000,0.1040,0.103,137.953995,4,alternative
1,5CwV9IOgmKbN7RdUTPBAYY,Smash Mouth,All Star,0,200373,False,0.731,0.867,11,-5.881,1,0.0320,0.03940,0.000000,0.0861,0.776,104.019997,4,alternative
2,3F4M4WsRWSIaHL2Nio1S8E,Smash Mouth,All Star,0,200373,False,0.731,0.867,11,-5.881,1,0.0320,0.03940,0.000000,0.0861,0.776,104.019997,4,alternative
3,7MUd9iu6pW3KnLcz2NuMcs,Jimmy Eat World,Last Christmas,0,267493,False,0.532,0.842,2,-6.152,1,0.0403,0.00131,0.000001,0.3300,0.715,126.042000,4,alternative
4,5AOrknyCsTI1KUBr92efWi,Chris Tomlin,Little Drummer Boy,0,215373,False,0.699,0.585,7,-5.755,1,0.0297,0.24600,0.000329,0.0975,0.757,124.054001,4,alternative


In [11]:
df.columns

Index(['track_id', 'artists', 'track_name', 'popularity', 'duration_ms',
       'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')

# Tasks to do

## 1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99


In [12]:
%%time
df1 = wr.athena.read_sql_query(sql="SELECT artists, track_name, popularity FROM {} WHERE popularity >= 99".format(table_name_csv), database=database_name)


CPU times: user 733 ms, sys: 38.9 ms, total: 772 ms
Wall time: 4.61 s


In [13]:
df1.head()

,artists,track_name,popularity
0,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
1,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


In [14]:
# Filter the DataFrame to include only rows where the popularity is 99 or greater
filtered_df = df[df['popularity'] >= 99]

# Select only the 'artists', 'track_name', and 'popularity' columns
result = filtered_df[['artists', 'track_name', 'popularity']]

# Display the result
result.head()


,artists,track_name,popularity
20001,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
77808,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


## 2. List artists with an average popularity of 92


In [15]:
%%time


df2 = wr.athena.read_sql_query(sql="SELECT artists, AVG(popularity) AS avg_popularity FROM {} GROUP BY artists HAVING AVG(popularity) = 92".format(table_name_csv), database=database_name)



CPU times: user 428 ms, sys: 45 ms, total: 473 ms
Wall time: 2.88 s


In [16]:
df2.head()


,artists,avg_popularity
0,Harry Styles,92.0
1,Rema;Selena Gomez,92.0


In [17]:
# Group the DataFrame by 'artists' and calculate the average popularity
grouped_df = df.groupby('artists')['popularity'].mean().reset_index()

# Filter the grouped DataFrame to include only those artists with an average popularity of 92
result = grouped_df[grouped_df['popularity'] == 92]

# Rename the columns for clarity
result.columns = ['artists', 'average_popularity']

# Display the result
print(result)


                 artists  average_popularity
11567       Harry Styles                92.0
22880  Rema;Selena Gomez                92.0


## 3. List the Top 10 most energetic genres


In [18]:
%%time
#df3 = wr.athena.read_sql_query(sql="SELECT AVG(energy) AS avg_energy, track_genre FROM {} GROUP BY track_genre ORDER BY avg_energy DESC LIMIT 10".format(table_name_csv), database=database_name)
df3 = wr.athena.read_sql_query(sql="SELECT track_genre, AVG(energy) AS avg_energy FROM {} GROUP BY track_genre ORDER BY avg_energy DESC LIMIT 10".format(table_name_csv), database=database_name)


CPU times: user 724 ms, sys: 36.5 ms, total: 761 ms
Wall time: 4.25 s


In [19]:
df3.head(10)

,track_genre,avg_energy
0,0.797,1174026.0
1,0.556,691306.0
2,0.492,542000.0
3,0.45,538160.0
4,0.347,526706.0
5,0.0761,502786.0
6,0.0903,449813.0
7,0.035,440310.0
8,0.483,371160.0
9,0.147,355693.0


In [20]:
# Group the DataFrame by 'track_genre' and calculate the average energy for each genre
grouped_df = df.groupby('track_genre')['energy'].mean().reset_index()

# Sort the results by average energy in descending order
sorted_df = grouped_df.sort_values(by='energy', ascending=False)

# Select the top 10 most energetic genres
top_genres = sorted_df.head(10)

# Display the result
top_genres


,track_genre,energy
123,0.797,1174026.0
103,0.556,691306.0
97,0.492,542000.0
91,0.45,538160.0
77,0.347,526706.0
24,0.0761,502786.0
28,0.0903,449813.0
8,0.035,440310.0
95,0.483,371160.0
43,0.147,355693.0


## 4. How many tracks is Bad Bunny on?

In [21]:
%%time
df4 = wr.athena.read_sql_query(sql="SELECT COUNT(*) AS bb_count FROM {} WHERE artists='Bad Bunny'".format(table_name_csv), database=database_name)


CPU times: user 400 ms, sys: 20.6 ms, total: 420 ms
Wall time: 2.93 s


In [22]:
df4.head()

,bb_count
0,48


In [23]:
# Filter the DataFrame for rows where 'artists' exactly matches 'Bad Bunny'
bad_bunny_tracks = df[df['artists'] == 'Bad Bunny']

# Count the number of such tracks
bad_bunny_count = bad_bunny_tracks.shape[0]

# Display the result
print(f"Bad Bunny is featured on {bad_bunny_count} tracks.")


Bad Bunny is featured on 48 tracks.


## 5. Show the top 10 genres in terms of popularity sorted by their most popular track


In [24]:
%%time

df5 = wr.athena.read_sql_query(sql="SELECT  track_genre, MAX(popularity) AS max_popularity FROM {} GROUP BY track_genre ORDER BY max_popularity DESC LIMIT 10".format(table_name_csv), database=database_name)


CPU times: user 630 ms, sys: 38.7 ms, total: 668 ms
Wall time: 2.97 s


In [25]:
df5.head(10)

,track_genre,max_popularity
0,pop,100
1,dance,100
2,latin,98
3,reggaeton,98
4,latino,98
5,edm,98
6,reggae,98
7,piano,96
8,rock,96
9,chill,93


In [26]:
# Group the DataFrame by 'track_genre' and find the maximum popularity for each genre
grouped_df = df.groupby('track_genre')['popularity'].max().reset_index()

# Sort the genres by the maximum popularity in descending order
sorted_df = grouped_df.sort_values(by='popularity', ascending=False)

# Select the top 10 entries
top_genres = sorted_df.head(10)

# Display the result
top_genres


,track_genre,popularity
679,pop,100
619,dance,100
688,reggaeton,98
629,edm,98
687,reggae,98
667,latino,98
666,latin,98
678,piano,96
689,rock,96
614,chill,93


## Release Resources

In [27]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>